 ## 📌 ACTIVIDAD 2: UN CASO DE REGRESIÓN MÁS REAL.


Crea el notebook saa_u03_p01_a2-<tus_iniciales>.ipynb donde entregar esta actividad. En el fichero
centro-comercial.csv tenemos registrada la facturación semanal conseguida en unos 45 centros
comerciales de una cadena de supermercados registrada un día concreto de cada semana. Tenemos
datos de varios años y queremos tener un modelo de Machine Learning que nos permita predecir las
ventas semanales a partir de otros datos que se indiquen.

 Nota: este ejercicio tiene un obstáculo que debes detectar para poder resolverlo de manera correcta (a mi
se me ocurren dos posibles soluciones aplicadas en la fase de preprocesamiento y en el ajuste de
parámetros, aunque solamente he probado una y es posible que haya más).

### COMPARATIVA DE VARIOS REGRESORES

Carga en un DataFrame el archivo centro-comercial.csv y realiza este preprocesamiento básico:
análisis exploratorio de datos y selección de características.

PRIMERA ETAPA: EDA.

 • Muestra unas filas del dataset y cuantas características y ejemplos tiene

 • Cambiar la característica fecha por 3 características nuevas: día de la semana, mes y año.
Puedes convertir las fechas en tipo Date con la función de pandas to_datetime(fecha,
formato) donde tendrás que indicar el formato porque utiliza por defecto el inglés. El formato
es un texto con comodines donde %d es el día, %m es el mes y %Y es el año con 4 dígitos. Cuando
la tengas convertida a tipo Date puedes extraerle información con expresiones como
pd.fecha.dt.year que te devuelve el año.

• Ahora cuenta y muestra la cantidad de diferentes valores que tiene
cada característica con df.nunique() para detectar posibles problemas
(que haya 13 meses por ejemplo) o detectar las que quizás sean
booleanas (tendrán 2 valores) y en definitiva comprender mejor los
datos.

 • Definimos una variable target que será "ventas_semanales" y otra de
tipo lista llamada predictoras que contenga el resto de características
menos el target.

 • Intentamos detectar qué características pueden ser pueden ser categóricas y cuales pueden ser
numéricas usando el criterio de la cantidad de valores diferentes que tengan. Si una
característica tiene menos del 0.5% de sus valores distintos es sospechosa de ser categórica
aunque sus valores sean numéricos. Completa el código que crea la lista nf con los nombres de
las características numéricas y la lista cf con las categóricas y las imprima:


In [ ]:
nu = df[predictoras].nunique().sort_values()
nf = []; cf = [];

 • Dibujar un histograma del target con seaborn por ejemplo usando histplot().


 • Completa este código que dibuja las distribuciones de las características numéricas para que
aparezca debajo del histograma de cada característica su boxplot y nos permitan visualizar la
presencia de outliers, tal y como se ve en la figura.

SEGUNDA ETAPA: SELECCIÓN DE CARACTERÍSTICAS.


 • Elimina características del dataframe que no son útiles, y también debes borrarlas de las listas
de predictoras y de cualquier otra lista donde aparezcan.

 • Realiza operaciones de limpieza que consideres necesarias. Crea un objeto llamado
preprocesador de tipo ColumnTransformer que pueda utilizarse para aplicar transformaciones
cuando se necesiten:

 ◦ Tratamiento de valores ausentes.

 ◦ Eliminación de filas repetidas.

 ◦ Tratamiento de outliers.

 ◦ Codificación de variables categóricas.

 ◦ Escalado o estandarización.

 • Estudia la colinealidad entre las predictoras y el target y entre cada pareja de predictoras
numéricas calculando la matriz de correlaciones de las características numéricas y visualizando
un mapa de calor de color azul y blanco. Elimina predictoras que presenten una correlación
superior al 60%.


TERCERA ETAPA: PARTICIONAR DATOS


 • Inicializa a partir de ahora todos los procesos aleatorios con una semilla aleatoria obtenida
como
<letras_de_tu_nombre>
concatenar
<letras_apellido1>
concatenar
<letras_apellido2>, por ejemplo en mi caso sería 449 (Jose = 4, Rosa=4, Rodríguez=9).


• Deja para entrenamiento el 80% de los datos y divide en train y test.

CUARTA ETAPA: ENTRENAMIENTO Y SELECCIÓN DE MODELOS

• Vamos a realizar esta etapa de forma manual. Definimos en un diccionario llamado regresores
todos los regresores que vamos a entrenar metidos en un Pipeline donde primero aplicamos el
preproceso a los datos y luego entrenamos/predecimos con el modelo.

In [ ]:
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Definir los diferentes regresores
regresores = {
    'svr (linear)': Pipeline([('pre', preprocesador), ('regresor', SVR(kernel='linear', C=1.0, epsilon=0.1))]),
    'svr (poly)': Pipeline([('pre', preprocesador), ('regresor', SVR(kernel='poly', C=1.0, epsilon=0.1))]),
    'svr (rbf)': Pipeline([('pre', preprocesador), ('regresor', SVR(kernel='rbf', C=1.0, epsilon=0.1))]),
    'nusvr': Pipeline([('pre', preprocesador), ('regresor', NuSVR(nu=0.5, kernel='rbf', C=1.0))]),
    'linearSVR': Pipeline([('pre', preprocesador), ('regresor', LinearSVR(epsilon=0.1, random_state=449))]),
    'linear Regression': Pipeline([('pre', preprocesador), ('regresor', LinearRegression())]),
    'Ridge': Pipeline([('pre', preprocesador), ('regresor', Ridge(alpha=1.0))]),
    'Lasso': Pipeline([('pre', preprocesador), ('regresor', Lasso(alpha=0.1))]),
    'Random Forest': Pipeline([('pre', preprocesador), ('regresor', RandomForestRegressor(n_estimators=100, random_state=449))]),
    'Gradient Boosting': Pipeline([('pre', preprocesador), ('regresor', GradientBoostingRegressor(n_estimators=100, random_state=449))]),
    'K-Neighbors': Pipeline([('pre', preprocesador), ('regresor', KNeighborsRegressor(n_neighbors=5))]),
    'Decision Tree': Pipeline([('pre', preprocesador), ('regresor', DecisionTreeRegressor(random_state=449))])
}

 • Ahora definimos listas (mse_train, mse_test, r2_train, r2_test) para registrar el error que
comete cada modelo con train y con test para poder comprobar el desempeño de cada uno y
detectar situaciones no deseables. Completa o adapta el código para generar salida como esta:


In [ ]:
# Entrenar y evaluar cada modelo
from sklearn.metrics import mean_squared_error, r2_score

mse_train = []
mse_test = []
r2_train = []
r2_test = []

plt.figure(figsize=(12, 10))

for i, (nombre, modelo) in enumerate(regresores.items()):
    modelo.fit(X_train, y_train_escalado) # modelo.fit(X_train, y_train) si ni el y_train ni el y_test estuviesen escalados
    y_pred_train = modelo.predict(X_train).reshape(-1, 1)
    y_pred_test = modelo.predict(X_test).reshape(-1, 1)
    # y_pred_test = s_y.inverse_transform(y_pred_test.reshape(-1, 1)).ravel() si el y_test no estuviese escalado

    mse_train.append(mean_squared_error(y_train_escalado, y_pred_train)) # mean_squared_error(y_train, y_pred_train)
    mse_test.append(mean_squared_error(y_test_escalado, y_pred_test))   # mean_squared_error(y_test, y_pred_test)
    r2_train.append(r2_score(y_train_escalado, y_pred_train))          # r2_score(y_train, y_pred_train)
    r2_test.append(r2_score(y_test_escalado, y_pred_test))            # r2_score(y_test, y_pred_test)

    plt.subplot(4, 3, i + 1)
    plt.scatter(X_test, y_test_escalado, color='red', label='Datos de test') # plt.scatter(X_test, y_test, color='red', label='Datos de test')
    plt.plot(X_test, y_pred_test, color='blue', linewidth=2, label=nombre)
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title(nombre)
    plt.legend()

plt.tight_layout()
plt.show()

 • Además de imprimir los scores (MSE y R2) para cada modelo y registrarlos en las listas, vas a
generar un gráfico donde se vea:

 ◦ Un scatter de color rojo que representa cada valor de y_test. Esto genera una línea recta de
color rojo formada por los valores de test: los puntos (y_test, y_test) siempre ocuparán
la diagonal del gráfico.

 ◦ Otro scatter de color azul representa cada valor de y_test y cada valor predicho para ese
y_test. Así podremos hacernos una idea de cómo de próximas están las predicciones del
regresor a sus valores reales.

 ◦ Puedes seleccionar uno de los gráficos del plt.figure(figsize=(12,10)) de la línea 7
ejecutando antes de crear los gráficos, la sentencia plt.subplot(5,3,i+1) dentro del bucle.

 • Fuera del bucle vamos a comparar el MSE de los modelos con un gráfico de barras donde se vea
train y test. Esto nos ayudará a detectar posibles underfitting y overfitting y ver desempeño de
los modelos.


 ENTREGA 4: Muestra:

 a) Etapas y pasos de la actividad con el texto, el código y los gráficos y resultados que se
generan en un notebook de Jupyter.

 b) Responde mirando tu gráfico de MSE:

 • El peor modelo parece ser:

• El mejor modelo parece ser:

• Cuál es el que tiene más overfitting:

•De manera similar haz un gráfico de barras donde aparezca el R2 de cada modelo.


ENTREGA 5: Responde:

 a) Responde mirando tu gráfico de R2:

 • El peor modelo y sus R2 de train y test:

• El mejor modelo y sus R2 de train y test:

• Cuál es el que tiene más overfitting:

•Escoge uno de los modelos de máquinas de soporte vectorial y ajusta sus hiperparámetros a mano
hasta que mejores su R2 con respecto el valor inicial.

 •Si K-NN tiene overfitting, intenta mejorar la configuración del modelo de la misma forma.

ENTREGA 6: Muestra:

 a) Nombre del modelo SVR que has escogido.

 b) MSE y R2 original del SVR:

c) Código que realice los cambios a la configuración de los modelos SVR y K-NN, los entrene y
calcule sus nuevos scores.

 d) Gráfico de puntos de datos de test y predicciones de ambos.

 e) MSE y R2 mejorado de ambos:
